In [5]:
import torch.nn.functional as F
import torchvision.transforms.functional as FT
from functools import partial
from torch import nn
from dataset import CocoDataset
from utils   import *
from model   import *
from metric  import *
from loss    import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup

Data transformations, dataset, & dataloader:

In [6]:
# define the sequence of transformations to apply to each image sample 
img_sz = 300
basic_tfs = [PhotometricDistort(1.),
             Flip(0.5),
             ImageToTensor(), CategoryToTensor(), BoxToTensor(),
             Zoomout(0.5, max_scale=2.5),
             Normalize(), 
             Resize((img_sz))]
tfms = transforms.Compose(basic_tfs)

# instantiate the dataset object
ds = CocoDataset(data_dir='./', dataset='val2017', anno_type='instances', transforms=tfms)

# create dataloader
BS = 8
dl = DataLoader(ds, batch_size=BS, shuffle=True, 
                collate_fn=partial(ds.collate_fn, img_resized=True)) # img_resized=true to indicate all image samples have been resized to same shape

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!


Model setup

In [8]:
# create the SSD model
ssd = SSD300(len(ds.id2cat))

Cost function

In [9]:
# multi-loss criteria
criterion = MultiBoxLoss(300, ssd.prior_boxes, threshold=0.5, neg_pos_ratio=3, alpha=1.)

Metric